In [2]:
search_for = 127
start_from = 0
threads = 10
thread = 8
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.004038810729980469
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 262209281
type: [1, 1, 1, 1, 127] 127
cases of this type: 260144641
100000 10.507127149362955
200000 18.721209932378144
300000 20.846292531556138
400000 19.798101463128713
500000 20.06693101820273
600000 20.561163900212694
700000 19.52313671007896
800000 19.89902753803957
900000 19.523448042452205
1000000 17.35364858758258
1100000 9.100863392896377
1200000 19.22969438956267
1300000 19.12333085527546
1400000 19.004724132547313
1500000 19.32601641477811
1600000 19.980275394157175
1700000 19.099388865533047
1800000 19.083834652340638
1900000 19.500776489171876
2000000 19.83949669869405
2100000 5.154864507256381
2200000 22.343117142612677
2300000 23.557116516321273
2400000 23.732816889225763
2500000 23.401606090514147
2600000 24.480291548677766
2700000 22.626603782648584
2800000 22.661451027759615
2900000 22.509579449343406
3000000 2

29400000 17.111506693344968
29500000 17.320313855634154
29600000 16.326113353025463
29700000 11.58634898198134
29800000 11.643391916317857
29900000 16.377253746761756
30000000 16.95040952562747
30100000 16.844193908993024
30200000 18.207534136206064
30300000 17.63031420671823
30400000 18.286966349558554
30500000 12.092381903508105
30600000 14.210572592684
30700000 18.16213881742686
30800000 18.69172574404249
30900000 18.469551776539163
31000000 9.671627689768394
31100000 16.481800463685
31200000 18.840319976213262
31300000 18.436061664229246
31400000 18.005784531911758
31500000 17.586061694687995
31600000 16.856330364774145
31700000 17.217039446420312
31800000 5.4200338428889
31900000 17.152451857674087
32000000 16.692211985180265
32100000 17.1999452364666
32200000 17.544273909566474
32300000 17.969613594514357
32400000 17.84315728573549
32500000 15.683400549319133
32600000 10.42788572044999
32700000 18.240834426662893
32800000 18.13448046894117
32900000 18.511143187915053
33000000 18.

59100000 14.941666521775383
59200000 15.343979561480987
59300000 15.232151594485373
59400000 15.586182229009102
59500000 16.005609505949273
59600000 15.494342494064671
59700000 15.670006329681053
59800000 15.184343585565658
59900000 8.36289237854528
60000000 12.742532822293903
60100000 13.818300391007044
60200000 14.139952540722986
60300000 13.79599031385981
60400000 12.826025787733498
60500000 6.704205556592032
60600000 13.829139369117096
60700000 13.804055336884442
60800000 13.524059664209156
60900000 13.570048550222051
61000000 8.260504813145841
61100000 13.737219952004219
61200000 15.106056415173656
61300000 15.036307244992354
61400000 15.587510930954707
61500000 15.833026923527166
61600000 15.571602411651698
61700000 15.25510033518904
61800000 15.17139473740665
61900000 15.011585043958108
62000000 6.213769256103085
62100000 13.689616366745351
62200000 13.368460253807683
62300000 13.338647247107184
62400000 13.238833262216652
62500000 6.613665046520356
62600000 12.52935647122918
62

88800000 6.877184831406737
88900000 9.948555561042065
89000000 12.077578322656867
89100000 8.424086233339041
89200000 7.814155700889679
89300000 11.488957228629502
89400000 11.065024654325848
89500000 6.832144693212741
89600000 11.40329568578806
89700000 11.53584630805405
89800000 11.877977711477284
89900000 12.410438635091852
90000000 12.786576303576252
90100000 12.603915370489792
90200000 12.734547772485646
90300000 12.627039491260929
90400000 12.534221683611014
90500000 12.272419903730624
90600000 12.048067738439622
90700000 11.485753806574605
90800000 11.989129234349859
90900000 5.372079745648937
91000000 11.377527943358727
91100000 11.366151504573814
91200000 3.8365308512623875
91300000 11.529749583452437
91400000 11.298086711732376
91500000 5.482230170315396
91600000 11.95731670657965
91700000 11.29534896494835
91800000 11.367814082946786
91900000 11.805685603876439
92000000 12.09028066560922
92100000 12.563875426616503
92200000 12.369057270966858
92300000 12.697835191928476
9240

118000000 8.801670104555889
118100000 9.552644202505254
118200000 9.570639575002142
118300000 9.547936819924148
118400000 9.749766185138284
118500000 9.661212644962744
118600000 9.819591032309807
118700000 9.908340733569508
118800000 10.47430938122854
118900000 10.678094230214697
119000000 9.439219282711482
119100000 9.453940699885868
119200000 9.117406540911544
119300000 9.114244819507384
119400000 9.154327049476853
119500000 9.291381079162308
119600000 8.824535433979941
119700000 9.280318434056703
119800000 3.0603714435919467
119900000 1.6960862672409793
120000000 6.596068040071619
120100000 9.014243924678837
120200000 8.956366957098501
120300000 9.204417006763753
120400000 9.1187261471745
120500000 9.242825189548757
120600000 9.006231035969094
120700000 9.471004091937338
120800000 9.416672605877912
120900000 11.102041547909584
121000000 10.678663331449908
121100000 10.198457335362226
121200000 10.00738751172195
121300000 9.777865352002102
121400000 10.044140726612593
121500000 10.04

146800000 7.641087342916571
146900000 7.879488199725992
147000000 8.162077119976201
147100000 7.946196537696886
147200000 7.975861139361281
147300000 7.865493469297982
147400000 8.4382923818505
147500000 8.587436391262399
147600000 8.665309274862173
147700000 8.136286142832441
147800000 7.717753211479817
147900000 7.889050516132766
148000000 7.899391568357218
148100000 7.6730657842326515
148200000 7.713107605115989
148300000 7.234637272934066
148400000 3.726606638152027
148500000 6.249204216935605
148600000 5.104410345239018
148700000 3.326440121634471
148800000 7.4953793245549525
148900000 7.626964033384063
149000000 7.773990527527052
149100000 7.800665677619553
149200000 7.7061678117208485
149300000 7.726158128728102
149400000 8.099263282565515
149500000 8.536658151488634
149600000 8.44171292898888
149700000 8.408603205979938
149800000 7.634009638328125
149900000 7.59597913526609
150000000 7.722497058500967
150100000 7.625969100610644
150200000 7.5349429772343095
150300000 7.49538129

176100000 6.40178428128585
176200000 6.499239458923035
176300000 6.264276439628005
176400000 6.346631355919901
176500000 6.07982511115506
176600000 6.039811712542831
176700000 5.6489911443464225
176800000 4.646123218994899
176900000 3.799211918535858
177000000 5.7099935004019144
177100000 5.714838243356194
177200000 3.1365151814422347
177300000 4.753334992348053
177400000 5.677289385462069
177500000 5.685550312259629
177600000 2.4579631992218536
177700000 5.751584305723788
177800000 5.652463742773969
177900000 6.247978511012745
178000000 6.052236231371491
178100000 6.252465460312392
178200000 6.276425982507962
178300000 6.247731170575186
178400000 6.164502575635914
178500000 6.0545633451237135
178600000 6.156869910777724
178700000 5.7641004141352115
178800000 5.610259302397207
178900000 2.687550236117018
179000000 5.722818445340653
179100000 5.765994707691445
179200000 5.474189760170759
179300000 2.340335246970034
179400000 5.624738164089982
179500000 5.870099310365914
179600000 4.9070

205300000 2.0241186016313413
205400000 4.39340824060366
205500000 4.025433423041984
205600000 3.806872852984294
205700000 4.144128549150181
205800000 3.8093546591785263
205900000 1.4772985417466309
206000000 4.281610215550249
206100000 3.829726748100168
206200000 4.125097780050807
206300000 3.9178964442148962
206400000 4.441294656234929
206500000 1.7329204079518536
206600000 4.372838086910391
206700000 4.681258749205178
206800000 4.305228200608229
206900000 4.575148571464947
207000000 4.3577056139434465
207100000 4.567545876614888
207200000 4.316637241909657
207300000 3.196781996775479
207400000 3.1438263143440897
207500000 3.980549484757173
207600000 4.2340928870202745
207700000 3.6768038609670564
207800000 4.036271967908597
207900000 2.9707177653667647
208000000 2.3285488520292024
208100000 4.114307450667082
208200000 3.7867384796156136
208300000 4.2459723430942065
208400000 4.03931991287871
208500000 3.7788119406022878
208600000 2.4246832272362675
208700000 4.192695885401233
2088000

234100000 2.0872825722961923
234200000 2.2966934635442433
234300000 2.0037608021582867
234400000 2.1301268118577044
234500000 1.9805119183632196
234600000 0.695875971553884
234700000 2.0947772404921703
234800000 1.9610408530322632
234900000 2.1846870963705465
235000000 2.049688268626279
235100000 2.2021494822620014
235200000 2.1337157629415135
235300000 2.1659789712380095
235400000 0.917007788489616
235500000 2.143600641226865
235600000 2.2960790451751185
235700000 2.0707428587864856
235800000 0.8997408993961049
235900000 2.173626156966041
236000000 2.0160640326475363
236100000 2.1284823576336627
236200000 1.9128378510388648
236300000 2.1244422657608273
236400000 1.9044185376693028
236500000 1.9829520538293248
236600000 1.0579843161409161
236700000 1.4857610389946598
236800000 1.9821502310672767
236900000 1.8893463299145121
237000000 2.133711079565295
237100000 1.9128803463533686
237200000 2.079345088459778
237300000 1.943468899847966
237400000 1.926330798382393
237500000 0.99722178552

262100000 0.0006819541499396165
best so far: 0
type: [1, 1, 127, 1, 1] 127
cases of this type: 16129
262200000 5.888433575762642e-05
best so far: 0
type: [1, 127, 1, 1, 1] 127
cases of this type: 127
best so far: 0
type: [127, 1, 1, 1, 1] 127
cases of this type: 1
best so far: 0
all cases: 262209281
best: 0
